# Fórmula Mágica - Joel Greenblat

Gerar um ranking de ações da Bovespa, seguinte critérios de liquidez diária, ROIC e P/L. Fonte dos dados: Fundamentus

## 1 - Importando Bibliotecas

In [1]:
import pandas as pd
import requests
import string

import warnings
warnings.filterwarnings('ignore')

## 2 - Carregando Dados

In [2]:
url = 'http://www.fundamentus.com.br/resultado.php'

header = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"  
 }

r = requests.get(url, headers=header)
df = pd.read_html(r.text,  decimal=',', thousands='.')[0]

# Pre-processamento dos dados: troca ',' por '.' em alguns decimais, e converte strings para float

for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  df[coluna] = df[coluna].str.replace('.', '')
  df[coluna] = df[coluna].str.replace(',', '.')
  df[coluna] = df[coluna].str.rstrip('%').astype(float) / 100

## Filtros de liquidez e "qualidade" - P/L e ROE

In [3]:
# Filtro de Liquidez: > 1 Milhão/2 meses
df = df[df['Liq.2meses'] > 1000000]

# Rankeamento - P/L (>0) e ROE
ranking = pd.DataFrame()
ranking['position'] = range(1,151)
ranking['P/L'] = df[df['P/L'] > 0].sort_values(by=['P/L'])['Papel'][:150].values
ranking['ROE'] = df.sort_values(by=['ROE'], ascending=False)['Papel'][:150].values

# Variaveis auxiliares para gerar ranking final - Imprime os 15 primeiros
aux_a = ranking.pivot_table(columns='P/L', values='position')
aux_b = ranking.pivot_table(columns='ROE', values='position')
aux_full=pd.concat([aux_a,aux_b])
Final_Rank = aux_full.dropna(axis=1).sum()
Final_Rank.sort_values()[:20]

MRFG3     11
CSNA3     35
BEEF3     38
TAEE11    41
TAEE3     42
TAEE4     43
TASA4     44
ETER3     46
TASA3     47
CYRE3     47
SULA11    51
TRPL4     52
CESP6     53
BRDT3     55
TIET11    56
RAPT4     58
TPIS3     59
WIZS3     60
GPCP3     61
CPLE3     66
dtype: int64

## Filtros de liquidez e "qualidade" alternativos - EV/EBIT e ROIC 

(DESCONSIDERAR FINANCEIRAS E BANCOS!!!)

In [4]:
# Filtro de Liquidez: > 1 Milhão/2 meses
df = df[df['Liq.2meses'] > 1000000]

# Rankeamento - EV/EBIT (>0) e ROIC
ranking = pd.DataFrame()
ranking['position'] = range(1,151)
ranking['EV/EBIT'] = df[df['EV/EBIT'] > 0].sort_values(by=['EV/EBIT'])['Papel'][:150].values
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values

# Variaveis auxiliares para gerar ranking final - Imprime os 15 primeiros
aux_a = ranking.pivot_table(columns='EV/EBIT', values='position')
aux_b = ranking.pivot_table(columns='ROIC', values='position')
aux_full=pd.concat([aux_a,aux_b])
Final_Rank = aux_full.dropna(axis=1).sum()
Final_Rank.sort_values()[:20]

PSSA3      6
WIZS3      8
VALE3     15
MRFG3     16
CMIN3     19
PLPL3     24
TIET11    39
CSNA3     40
BEEF3     43
ALUP11    49
TASA4     49
TASA3     51
AURA33    51
TRPL4     57
JBSS3     61
USIM5     62
USIM3     65
PCAR3     66
SAPR11    67
SAPR3     67
dtype: int64